In [98]:
import pandas as pd
import numpy as np

path_weather = r'C:\Users\20177480\Desktop\Case Studies\Heijmans Infra (Modeling and Simulation)\Data\Weather.xlsx'
path_energy = r'C:\Users\20177480\Desktop\Case Studies\Heijmans Infra (Modeling and Simulation)\Data\Energydata.xlsx'
df_temperature =pd.read_excel(path_weather)
df_energy =pd.read_excel(path_energy)

df_temperature['timestamp'] = pd.to_datetime(df_temperature['timestamp'], format="%d-%m-%Y %H:%M.%S")
df_temperature = df_temperature[df_temperature['timestamp']>'23-10-2017 09:00:00']
del df_temperature['humidity']
df_temperature = df_temperature.set_index('timestamp')

df_energy['Time'] = pd.to_datetime(df_energy['Time'], format="%d-%m-%Y %H:%M.%S")
df_energy = df_energy[df_energy['Time']>'23-10-2017 09:00:00']
df_energy['Energy'] = df_energy['Energy kWh']
del df_energy['Energy kWh']
df_energy = df_energy.set_index('Time')

In [99]:
df_features = df_temperature.resample('D').mean()
df_features['maximum_temp'] = df_temperature.resample('D').max()
df_features['minimum_temp'] = df_temperature.resample('D').min()
df_features_energy = df_energy.resample('D').mean()
df_features_energy['maximum_energy'] = df_energy.resample('D').max()
df_features_energy['total_energy'] = df_energy.resample('D').sum()

In [100]:
features = pd.concat([df_features, df_features_energy], axis=1)
features.head()

,temperature,maximum_temp,minimum_temp,Energy,maximum_energy,total_energy
timestamp,,,,,,
2017-10-23,13.229464,15.1,11.3,8.444444,11,152
2017-10-24,15.043333,16.9,13.0,13.239583,24,1271
2017-10-25,14.662222,16.7,9.7,13.312500,22,1278
2017-10-26,14.000000,17.6,11.7,13.260417,22,1273
2017-10-27,11.470556,13.9,8.9,11.437500,21,1098


In [105]:
def df_autocorr(df, lag=7, axis=0):
    return df.apply(lambda col: col.autocorr(lag), axis=axis)
print df_autocorr(features)

temperature       0.366369
maximum_temp      0.287585
minimum_temp      0.365371
Energy            0.703744
maximum_energy    0.677275
total_energy      0.618777
dtype: float64


### Pearson correlation is for linearly related variables

In [102]:
import scipy
print scipy.stats.mstats.normaltest(features, axis=0)
corr_matrix_pearson = features.corr(method='pearson')

NormaltestResult(statistic=masked_array(data = [0.07164115118179298 0.09529319844915315 0.2978407032582245
 86.1722028838809 136.80445786481687 8.649773532344716],
             mask = [False False False False False False],
       fill_value = 1e+20)
, pvalue=array([  9.64813389e-01,   9.53470685e-01,   8.61637740e-01,
         1.94063517e-19,   1.96466915e-30,   1.32350487e-02]))


In [103]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

trace = go.Heatmap(z=corr_matrix_pearson.values,
                   x=['temperature', 'maximum_temp', 'minimum_temp', 'Energy', 'maximum_energy','total_energy'],
                   y=['temperature', 'maximum_temp', 'minimum_temp', 'Energy', 'maximum_energy','total_energy'])
data=[trace]

fig = go.Figure(data=data)
plot(fig, filename='basic-heatmap.html')

'file://C:\\Users\\20177480\\Documents\\GitHub\\Heijmans - NMM\\modelling_and_simulation\\basic-heatmap.html'

In [104]:
corr_matrix_kendall = features.corr(method='kendall')
print corr_matrix_kendall

                temperature  maximum_temp  minimum_temp    Energy  \
temperature        1.000000      0.797527      0.828580  0.139490   
maximum_temp       0.797527      1.000000      0.647418  0.126954   
minimum_temp       0.828580      0.647418      1.000000  0.128807   
Energy             0.139490      0.126954      0.128807  1.000000   
maximum_energy    -0.010147      0.004526     -0.026257  0.694792   
total_energy       0.136383      0.133751      0.118789  0.967718   

                maximum_energy  total_energy  
temperature          -0.010147      0.136383  
maximum_temp          0.004526      0.133751  
minimum_temp         -0.026257      0.118789  
Energy                0.694792      0.967718  
maximum_energy        1.000000      0.676669  
total_energy          0.676669      1.000000  


In [52]:
corr_matrix_spearman = features.corr(method='spearman')

In [53]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

trace = go.Heatmap(z=corr_matrix_spearman.values,
                   x=['temperature', 'maximum_temp', 'minimum_temp', 'Energy', 'maximum_energy','minimum_energy','total_energy'],
                   y=['temperature', 'maximum_temp', 'minimum_temp', 'Energy', 'maximum_energy','minimum_energy','total_energy'])
data=[trace]

fig = go.Figure(data=data)
plot(fig, filename='basic-heatmap.html')

'file://C:\\Users\\20177480\\Documents\\GitHub\\Heijmans - NMM\\modelling_and_simulation\\basic-heatmap.html'